In [3]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.5f}'.format

In [4]:
previous_df = pd.read_csv('previous_application.csv')

POS_CASH_df = pd.read_csv('POS_CASH_merge.csv')
installments_df = pd.read_csv('installments_merge.csv')
credit_card_df = pd.read_csv('credit_card_merge.csv')

In [5]:
merged_previous = pd.merge(previous_df, POS_CASH_df, on = 'SK_ID_PREV', how = 'left')
merged_previous = pd.merge(merged_previous, installments_df, on = 'SK_ID_PREV', how = 'left')
merged_previous = pd.merge(merged_previous, credit_card_df, on = 'SK_ID_PREV', how = 'left')

In [6]:
merged_previous

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,CNT_INSTALMENT_MATURE_CUM_STD,CNT_INSTALMENT_MATURE_CUM_FIRST,SK_DPD_MEAN_y,SK_DPD_MEDIAN_y,SK_DPD_STD_y,SK_DPD_FIRST_y,SK_DPD_DEF_MEAN_y,SK_DPD_DEF_MEDIAN_y,SK_DPD_DEF_STD_y,SK_DPD_DEF_FIRST_y
0,2030495,271877,Consumer loans,1730.43000,17145.00000,17145.00000,0.00000,17145.00000,SATURDAY,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2802425,108129,Cash loans,25188.61500,607500.00000,679671.00000,NaN,607500.00000,THURSDAY,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2523466,122040,Cash loans,15060.73500,112500.00000,136444.50000,NaN,112500.00000,TUESDAY,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2819243,176158,Cash loans,47041.33500,450000.00000,470790.00000,NaN,450000.00000,MONDAY,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1784265,202054,Cash loans,31924.39500,337500.00000,404055.00000,NaN,337500.00000,THURSDAY,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.29000,267295.50000,311400.00000,0.00000,267295.50000,WEDNESDAY,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1670210,2357031,334635,Consumer loans,6622.02000,87750.00000,64291.50000,29250.00000,87750.00000,TUESDAY,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1670211,2659632,249544,Consumer loans,11520.85500,105237.00000,102523.50000,10525.50000,105237.00000,MONDAY,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1670212,2785582,400317,Cash loans,18821.52000,180000.00000,191880.00000,NaN,180000.00000,WEDNESDAY,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def object_pre (df) :
    
    pk_id = sorted(list(df['SK_ID_CURR'].unique()))
    first_id = []
    mode_id = []
    object_col_first = []
    object_col_mode = []

    for col in df:
        if df[col].dtype == 'object':
            first_id.append(col+'_FIRST')
            mode_id.append(col+'_MODE')
            print(col)
            object_col_first.append(list(df.groupby('SK_ID_CURR')[col].first()))
            object_col_mode.append(list(df.groupby('SK_ID_CURR')[col].apply(lambda x : x.mode()[0])))

    object_col_first_df = pd.DataFrame(object_col_first).transpose()
    object_col_mode_df = pd.DataFrame(object_col_mode).transpose()   

    object_col_first_df.columns = [first_id]
    object_col_mode_df.columns = [mode_id]
    return pk_id, object_col_first_df, object_col_mode_df

In [8]:
temp_merged_previous = merged_previous.copy()

for col in temp_merged_previous:
    if temp_merged_previous[col].dtype == 'object':
        temp_merged_previous[col] = temp_merged_previous[col].fillna('NULL')

In [9]:
pk_id, object_col_first, object_col_mode = object_pre(temp_merged_previous)

NAME_CONTRACT_TYPE
WEEKDAY_APPR_PROCESS_START
FLAG_LAST_APPL_PER_CONTRACT
NAME_CASH_LOAN_PURPOSE
NAME_CONTRACT_STATUS
NAME_PAYMENT_TYPE
CODE_REJECT_REASON
NAME_TYPE_SUITE
NAME_CLIENT_TYPE
NAME_GOODS_CATEGORY
NAME_PORTFOLIO
NAME_PRODUCT_TYPE
CHANNEL_TYPE
NAME_SELLER_INDUSTRY
NAME_YIELD_GROUP
PRODUCT_COMBINATION
NAME_CONTRACT_STATUS_FIRST_x
NAME_CONTRACT_STATUS_MODE_x
NAME_CONTRACT_STATUS_FIRST_y
NAME_CONTRACT_STATUS_MODE_y


In [10]:
merge_obj_df = pd.concat([object_col_first, object_col_mode], axis = 1)
merge_obj_df.index = pk_id
merge_obj_df = merge_obj_df.reset_index()
merge_obj_df = merge_obj_df.rename(columns={'index':'SK_ID_CURR'})
merge_obj_df = merge_obj_df.replace('NULL', np.nan)
merge_obj_df.columns = ['SK_ID_CURR', 'NAME_CONTRACT_TYPE_FIRST', 'WEEKDAY_APPR_PROCESS_START_FIRST', 'FLAG_LAST_APPL_PER_CONTRACT_FIRST', \
'NAME_CASH_LOAN_PURPOSE_FIRST', 'NAME_CONTRACT_STATUS_FIRST', 'NAME_PAYMENT_TYPE_FIRST', 'CODE_REJECT_REASON_FIRST', \
'NAME_TYPE_SUITE_FIRST', 'NAME_CLIENT_TYPE_FIRST', 'NAME_GOODS_CATEGORY_FIRST', 'NAME_PORTFOLIO_FIRST', 'NAME_PRODUCT_TYPE_FIRST', \
'CHANNEL_TYPE_FIRST', 'NAME_SELLER_INDUSTRY_FIRST', 'NAME_YIELD_GROUP_FIRST', 'PRODUCT_COMBINATION_FIRST', 'NAME_CONTRACT_STATUS_FIRST_x_FIRST', \
'NAME_CONTRACT_STATUS_MODE_x_FIRST', 'NAME_CONTRACT_STATUS_FIRST_y_FIRST', 'NAME_CONTRACT_STATUS_MODE_y_FIRST', 'NAME_CONTRACT_TYPE_MODE', \
'WEEKDAY_APPR_PROCESS_START_MODE', 'FLAG_LAST_APPL_PER_CONTRACT_MODE', 'NAME_CASH_LOAN_PURPOSE_MODE', 'NAME_CONTRACT_STATUS_MODE', \
'NAME_PAYMENT_TYPE_MODE', 'CODE_REJECT_REASON_MODE', 'NAME_TYPE_SUITE_MODE', 'NAME_CLIENT_TYPE_MODE', 'NAME_GOODS_CATEGORY_MODE', \
'NAME_PORTFOLIO_MODE', 'NAME_PRODUCT_TYPE_MODE', 'CHANNEL_TYPE_MODE', 'NAME_SELLER_INDUSTRY_MODE', 'NAME_YIELD_GROUP_MODE', \
'PRODUCT_COMBINATION_MODE', 'NAME_CONTRACT_STATUS_FIRST_x_MODE', 'NAME_CONTRACT_STATUS_MODE_x_MODE', \
'NAME_CONTRACT_STATUS_FIRST_y_MODE', 'NAME_CONTRACT_STATUS_MODE_y_MODE']
merge_obj_df

,SK_ID_CURR,NAME_CONTRACT_TYPE_FIRST,WEEKDAY_APPR_PROCESS_START_FIRST,FLAG_LAST_APPL_PER_CONTRACT_FIRST,NAME_CASH_LOAN_PURPOSE_FIRST,NAME_CONTRACT_STATUS_FIRST,NAME_PAYMENT_TYPE_FIRST,CODE_REJECT_REASON_FIRST,NAME_TYPE_SUITE_FIRST,NAME_CLIENT_TYPE_FIRST,...,NAME_PORTFOLIO_MODE,NAME_PRODUCT_TYPE_MODE,CHANNEL_TYPE_MODE,NAME_SELLER_INDUSTRY_MODE,NAME_YIELD_GROUP_MODE,PRODUCT_COMBINATION_MODE,NAME_CONTRACT_STATUS_FIRST_x_MODE,NAME_CONTRACT_STATUS_MODE_x_MODE,NAME_CONTRACT_STATUS_FIRST_y_MODE,NAME_CONTRACT_STATUS_MODE_y_MODE
0,100001,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,Refreshed,...,POS,XNA,Country-wide,Connectivity,high,POS mobile with interest,Completed,Active,NaN,NaN
1,100002,Consumer loans,SATURDAY,Y,XAP,Approved,XNA,XAP,NaN,New,...,POS,XNA,Stone,Auto technology,low_normal,POS other with interest,Active,Active,NaN,NaN
2,100003,Cash loans,FRIDAY,Y,XNA,Approved,XNA,XAP,Unaccompanied,Repeater,...,POS,XNA,Country-wide,Consumer electronics,middle,Cash X-Sell: low,Active,Active,NaN,NaN
3,100004,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,New,...,POS,XNA,Regional / Local,Connectivity,middle,POS mobile without interest,Active,Active,NaN,NaN
4,100005,Cash loans,FRIDAY,Y,XNA,Canceled,XNA,XAP,NaN,Repeater,...,POS,XNA,Country-wide,Connectivity,XNA,Cash,Active,Active,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338852,456251,Consumer loans,THURSDAY,Y,XAP,Approved,Cash through the bank,XAP,NaN,New,...,POS,XNA,Country-wide,Connectivity,high,POS mobile with interest,Signed,Active,NaN,NaN
338853,456252,Consumer loans,WEDNESDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,New,...,POS,XNA,Country-wide,Consumer electronics,low_normal,POS household without interest,Completed,Active,NaN,NaN
338854,456253,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,Refreshed,...,POS,XNA,Stone,Connectivity,high,POS mobile with interest,Active,Active,NaN,NaN
338855,456254,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,Repeater,...,POS,XNA,Country-wide,Connectivity,high,POS household with interest,Active,Active,NaN,NaN


In [11]:
level_0 = []

for col in merged_previous:
    if merged_previous[col].dtype != 'object':
        level_0.append(col)

In [12]:
level_1 = ['_MEAN', '_MEDIAN', '_STD', '_FIRST']

col_names = []
for name1 in level_0[2:] :
    for name2 in level_1 :
        col_names.append(name1+name2)

In [13]:
merge_real_num_df = pd.DataFrame(merged_previous.groupby('SK_ID_CURR').agg(['mean','median','std','first']))
merge_real_num_df.reset_index(drop = True, inplace = True)
merge_real_num_df.columns = merge_real_num_df.columns.droplevel(0)
merge_real_num_df = merge_real_num_df.iloc[:,4:]
merge_real_num_df.columns = col_names
merge_real_num_df

,AMT_ANNUITY_MEAN,AMT_ANNUITY_MEDIAN,AMT_ANNUITY_STD,AMT_ANNUITY_FIRST,AMT_APPLICATION_MEAN,AMT_APPLICATION_MEDIAN,AMT_APPLICATION_STD,AMT_APPLICATION_FIRST,AMT_CREDIT_MEAN,AMT_CREDIT_MEDIAN,...,SK_DPD_DEF_MEDIAN_y_STD,SK_DPD_DEF_MEDIAN_y_FIRST,SK_DPD_DEF_STD_y_MEAN,SK_DPD_DEF_STD_y_MEDIAN,SK_DPD_DEF_STD_y_STD,SK_DPD_DEF_STD_y_FIRST,SK_DPD_DEF_FIRST_y_MEAN,SK_DPD_DEF_FIRST_y_MEDIAN,SK_DPD_DEF_FIRST_y_STD,SK_DPD_DEF_FIRST_y_FIRST
0,3951.00000,3951.00000,NaN,3951.00000,24835.50000,24835.50000,NaN,24835.50000,23787.00000,23787.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9251.77500,9251.77500,NaN,9251.77500,179055.00000,179055.00000,NaN,179055.00000,179055.00000,179055.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56553.99000,64567.66500,46332.55778,98356.99500,435436.50000,337500.00000,424161.62055,900000.00000,484191.00000,348637.50000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5357.25000,5357.25000,NaN,5357.25000,24282.00000,24282.00000,NaN,24282.00000,20106.00000,20106.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4813.20000,4813.20000,NaN,4813.20000,22308.75000,22308.75000,31549.33681,0.00000,20076.75000,20076.75000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338852,6605.91000,6605.91000,NaN,6605.91000,40455.00000,40455.00000,NaN,40455.00000,40455.00000,40455.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338853,10074.46500,10074.46500,NaN,10074.46500,57595.50000,57595.50000,NaN,57595.50000,56821.50000,56821.50000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338854,4770.40500,4770.40500,1127.56662,5567.71500,24162.75000,24162.75000,6717.16087,28912.50000,20625.75000,20625.75000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338855,10681.13250,10681.13250,11857.74585,19065.82500,121317.75000,121317.75000,144916.93861,223789.50000,134439.75000,134439.75000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
total_merged_previous_df = pd.concat([merge_obj_df, merge_real_num_df], axis = 1)
total_merged_previous_df

,SK_ID_CURR,NAME_CONTRACT_TYPE_FIRST,WEEKDAY_APPR_PROCESS_START_FIRST,FLAG_LAST_APPL_PER_CONTRACT_FIRST,NAME_CASH_LOAN_PURPOSE_FIRST,NAME_CONTRACT_STATUS_FIRST,NAME_PAYMENT_TYPE_FIRST,CODE_REJECT_REASON_FIRST,NAME_TYPE_SUITE_FIRST,NAME_CLIENT_TYPE_FIRST,...,SK_DPD_DEF_MEDIAN_y_STD,SK_DPD_DEF_MEDIAN_y_FIRST,SK_DPD_DEF_STD_y_MEAN,SK_DPD_DEF_STD_y_MEDIAN,SK_DPD_DEF_STD_y_STD,SK_DPD_DEF_STD_y_FIRST,SK_DPD_DEF_FIRST_y_MEAN,SK_DPD_DEF_FIRST_y_MEDIAN,SK_DPD_DEF_FIRST_y_STD,SK_DPD_DEF_FIRST_y_FIRST
0,100001,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,Refreshed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,Consumer loans,SATURDAY,Y,XAP,Approved,XNA,XAP,NaN,New,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100003,Cash loans,FRIDAY,Y,XNA,Approved,XNA,XAP,Unaccompanied,Repeater,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100004,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,New,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005,Cash loans,FRIDAY,Y,XNA,Canceled,XNA,XAP,NaN,Repeater,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338852,456251,Consumer loans,THURSDAY,Y,XAP,Approved,Cash through the bank,XAP,NaN,New,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338853,456252,Consumer loans,WEDNESDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,New,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338854,456253,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,Refreshed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338855,456254,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,Repeater,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
total_merged_previous_df.to_csv('previous_merge.csv')